In [ ]:
import numpy as np
from skimage import io, feature
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from deap import algorithms, base, creator, tools

# Cargar imágenes y etiquetas
images = io.imread_collection("mammography/*.png")
labels = np.loadtxt("labels.txt")

# Preprocesamiento de imágenes
X = []
for img in images:
    # Aplicar un filtro de suavizado gaussiano para reducir el ruido
    img = feature.denoise_bilateral(img, sigma_range=0.05, sigma_spatial=15)
    # Aplicar un filtro de mejora de contraste para mejorar la visibilidad de los tumores
    img = feature.equalize_adapthist(img, clip_limit=0.03)
    X.append(img.ravel())
X = np.array(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Definir la función de aptitud para el algoritmo genético
def evaluate(individual):
    # Seleccionar las características más importantes utilizando el índice binario
    X_train_selected = X_train[:, individual]
    X_test_selected = X_test[:, individual]
    # Entrenar un clasificador de Random Forest en los datos seleccionados
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train_selected, y_train)
    # Calcular la precisión del clasificador en el conjunto de prueba
    accuracy = clf.score(X_test_selected, y_test)
    return accuracy,

# Definir el tipo de aptitud y el tipo de individuo para el algoritmo genético
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

# Definir la población inicial y el algoritmo genético
toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X_train[0]))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
pop = toolbox.population(n=100)
algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10)

# Seleccionar las características más importantes utilizando el mejor individuo
best_ind = tools.selBest(pop, k=1)[0]
X_train_selected = X_train[:, best_ind]
X_test_selected = X_test[:, best_ind]

# Entrenar y evaluar